# Introduction to Generator Network - Text Script Generation
In this project, we'll generate our own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  We'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network we'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).


In [1]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, we first need to transform the words to ids.  

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    unique_words = set([txt.lower() for txt in text] )
    vocab_to_int = {word:idx for idx,word in enumerate(unique_words)}
    int_to_vocab = {idx:word for idx,word in enumerate(unique_words)}

    return vocab_to_int, int_to_vocab


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure not to use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    punc_dict = {'.':'||Period||',
                 ',': '||Comma||', 
                 '"':'||QuotationMark||',
                 ';':'||Semicolon||',
                 '!':'||Exclamationmark||',
                 '?':'||Questionmark||',
                 '(':'||LeftParentheses||',
                 ')':'||RightParentheses||',
                 '--':'||Dash||',
                 '\n':'||Return||'}
    return punc_dict

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
import helper
import numpy as np
import problem_unittests as tests
import tensorflow as tf

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
We'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches



### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input_text = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None), name='targets')
    learning_rate = tf.placeholder(tf.float32, None, name='LearningRate')
    return input_text, targets, learning_rate


tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm] * 1)
    
    initialize_size = stacked_lstm.zero_state(batch_size, dtype=tf.float32)
    initialize_size = tf.identity(initialize_size, name='initial_state')
    return stacked_lstm, initialize_size


tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embeddings = tf.Variable(tf.truncated_normal([vocab_size, embed_dim]))
    embed = tf.nn.embedding_lookup(embeddings, input_data)
    return embed


tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
We created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, states = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    states = tf.identity(states, name='final_state')
    return outputs, states


tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions we implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data,vocab_size,rnn_size)
    outputs, states = build_rnn(cell,embed)
    predictions = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=None)
    return predictions, states


tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If we can't fill the last batch with enough data, we need to drop it.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    index=0
    sequences_input=[]
    sequences_output=[]
    for x in range(int(len(int_text)/seq_length)):
        sequences_input.append(int_text[index:index+seq_length])
        sequences_output.append(int_text[index+1:index+seq_length+1])
        if(index+seq_length+1>len(int_text)):
            for i in range(len(int_text)-(index+seq_length+1)):
                sequences_output.append(0)
        index+=seq_length
    batches_input=[]
    batches_targets=[]
    batches=[]
    batch={}
    group_ind=int(len(sequences_input)/batch_size)
    for x in range(group_ind):    
        batches.append([sequences_input[x*group_ind:x*group_ind+batch_size],sequences_output[x*group_ind:x*group_ind+batch_size]])
    return np.array(batches)

    '''index=0
    sequences_input=[]
    sequences_output=[]
    #print(len(int_text))
    for x in range(int(len(int_text)/seq_length)):
        sequences_input.append(int_text[index:index+seq_length])
        sequences_output.append(int_text[index+1:index+seq_length+1])
        index+=seq_length
    #print(sequences_output)
    batches_input=[]
    batches_targets=[]
    batches=[]
    group_ind=int(len(sequences_input)/batch_size)
    sequences_input = sequences_input[:batch_size*group_ind]
    sequences_output = sequences_output[:batch_size*group_ind]
    for x in range(int(len(sequences_input)/batch_size)): 
        batches_input=sequences_input[x::group_ind]
        batches_output=sequences_output[x::group_ind]
        comp= [batches_input, batches_output]
        batches.append(comp)
    #print(np.array(batches).shape)        
    return np.array(batches)'''


tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [15]:
# Number of Epochs
num_epochs = 500
# Batch Size
batch_size = 64
# RNN Size
# Embedding Dimension Size
embed_dim = None
rnn_size = 512
# Sequence Length
seq_length = 32
# Learning Rate
learning_rate = 0.0001
# Show stats for every n number of batches
show_every_n_batches = 20

save_dir = './save'

### Build the Graph
Build the graph using the neural network we implemented.

In [16]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  

In [20]:

batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.822
Epoch   0 Batch   20/33   train_loss = 8.497
Epoch   1 Batch    7/33   train_loss = 6.997
Epoch   1 Batch   27/33   train_loss = 6.179
Epoch   2 Batch   14/33   train_loss = 6.052
Epoch   3 Batch    1/33   train_loss = 5.912
Epoch   3 Batch   21/33   train_loss = 5.826
Epoch   4 Batch    8/33   train_loss = 5.872
Epoch   4 Batch   28/33   train_loss = 5.767
Epoch   5 Batch   15/33   train_loss = 5.825
Epoch   6 Batch    2/33   train_loss = 5.808
Epoch   6 Batch   22/33   train_loss = 5.410
Epoch   7 Batch    9/33   train_loss = 5.550
Epoch   7 Batch   29/33   train_loss = 5.423
Epoch   8 Batch   16/33   train_loss = 5.407
Epoch   9 Batch    3/33   train_loss = 5.498
Epoch   9 Batch   23/33   train_loss = 5.196
Epoch  10 Batch   10/33   train_loss = 5.382
Epoch  10 Batch   30/33   train_loss = 5.226
Epoch  11 Batch   17/33   train_loss = 5.215
Epoch  12 Batch    4/33   train_loss = 5.132
Epoch  12 Batch   24/33   train_loss = 5.145
Epoch  13 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [21]:

# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [22]:

import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [23]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    with loaded_graph.as_default():
        input_tensor = loaded_graph.get_tensor_by_name('input:0')
        initial_state=loaded_graph.get_tensor_by_name('initial_state:0')
        final_state=loaded_graph.get_tensor_by_name('final_state:0')
        probs=loaded_graph.get_tensor_by_name('probs:0')
        
    return input_tensor, initial_state, final_state, probs


tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [38]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    max_prob = max(probabilities)
    idx = [i for i, j in enumerate(probabilities) if j == max_prob]
    return np.random.choice(list(int_to_vocab.values()),1,p=probabilities)[0]


tests.test_pick_word(pick_word)

Tests Passed


## Generate the Script
This will generate the script for us.  Set `gen_length` to the length of TV script you want to generate.

In [39]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: they wanted it more.
barney_gumble: hey homer, didn't you say if duff dry wins, your daughter loves you?
homer_simpson:(aggravated) not duff dry.
barney_gumble: wow! i haven't seen barney since you were one of it!
moe_szyslak: hundred and ten, a little old man, me and he never get one. now don't blame me.
homer_simpson: the last angry clown, the car.
patty_bouvier: in the bar, moe. then he looks like a good.
moe_szyslak: wait a minute. i'll straighten ya.
grampa_simpson: yeah, you shouldn't be some kind of life.
seymour_skinner:(turns to burn) you your old man--(looking at marge) what do you doin'? i'm gonna do as a guy like?
moe_szyslak: well, the greatest!
barney_gumble: but on the little miss springfield by you for the fire truck to make a creature was a physical attraction.
homer_simpson:(weary) oh man, a little man. maybe i work my love with my new air conditioner.


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, we'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the beginnning of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  
